# Projet Machine Learning - L3 Groupe A - 08/05/2023
## M. Chantot - P. Bouzard - C. Brun 

In [48]:
import numpy as np # Algèbre liéaire
import pandas as pd # Data rocessig
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#from scipy import spearmanr

In [49]:
# Montage local à partir de Google Drive
# Importation des fichiers .csv
# Chemin d'accès aux données dans Google Drive : Placez les données dans un fichier Data dans votre Google Drive
Data_X_path = './Data_X.csv'
DataNew_X_path = './DataNew_X.csv'
Data_Y_path = './Data_Y.csv'

# Lecture des fichiers csv avec pandas
dfX = pd.read_csv(Data_X_path)
dfNX = pd.read_csv(DataNew_X_path)
dfY = pd.read_csv(Data_Y_path)

In [50]:
# Abandon de l'attribut "COUNTRY" qui ne nous servira pas.
dfNX = dfNX.drop(['COUNTRY'], axis=1)

# Obtention de numpy à partir d'un dataframe pandas
Data_X = dfX
DataNew_X = dfNX
Data_Y = dfY
# Affichage des informations des données importées
dfX.info()
dfNX.info()
dfY.info()

In [51]:
# Copie des données pour éviter de détruire les fichiers d'origine
Data_X = dfX.copy()
Data_NX = dfNX.copy()
Data_Y = dfY.copy()

# Affichage des 5 premières lignes pour voir si cela a fonctionné
Data_X.head()

In [52]:
Data_NX.head()

In [53]:
Data_Y.head()

## La première étape consiste à nettoyer le jeu de données
On retire pour cela les données manquantes, les doublons et les valeurs aberrantes

In [54]:
# Vérifier s’il y a des valeurs manquantes dans les données.
# Retirer les lignes avec des valeurs manquantes, et les lignes de prix correspondantes
# Pour cela, on commence par concaténer les deux jeux de données
XY = pd.concat([Data_X,Data_Y], axis=1)
XY = XY.drop(['COUNTRY'], axis=1) # On supprime la colonne COUNTRY comme les données à l'interieur sont de STRING, n'étant alors pas intereant à étudier
print("Data longueur : ", len(XY))
# On retire les lignes avec des valeurs manquantes
XY = XY.dropna()
DataNew_X = DataNew_X.dropna()
print("Data longueur sans les lignes manquantes: ", len(XY))
# On retire les lignes dupliquées
XY = XY.drop_duplicates()
DataNew_X = DataNew_X.drop_duplicates()
print("Data longueur sans les lignes dupliquées: ", len(XY))
# On cherche maintenant à retirer les valeurs aberrantes
print("Ecart type des valeurs de XY :")
print(XY.std(numeric_only = True)) # Ecart type avant les valeurs aberrantes
XY = XY[(np.abs(stats.zscore(XY)) < 3).all(axis=1)]
DataNew_X = DataNew_X[(np.abs(stats.zscore(DataNew_X)) < 3).all(axis=1)]
print("Data longueur sans les valeurs aberrantes: ", len(XY))
# On affiche les valeurs moyennes et les écarts types
print("Moyenne des valeurs de XY :")
print(XY.mean(numeric_only = True))
print("Ecart type des valeurs de XY :")
print(XY.std(numeric_only = True)) # Ecart type après les valeurs aberrantes
# On sépare ensuite les deux jeux de données
Data_X = XY.iloc[:,:-2]
Data_Y = XY.iloc[:,-2:]
'''
print("X, : \n", Data_X.head())
print("Y : \n", Data_Y.head())
'''
# Vérifier si les valeurs des différents attributs sont comparables

De ce que nous voyons au-dessus, de nombreuses valeurs semblent corrélées, cependant il n'est pas forcément juste de les mettre en relation. Par exemple, mettre en relation les données lié à l'Allemagne entre elle paraît sensé, et comparer certaines mesures entre les deux pays à du sens, alors que de comparé majoritairement les valeurs d'un pays a l'autre ne paraît pas être la meilleure comparaison possible.

## Ensuite on prépare le jeu de données
Pour cela on procède à une standardisation des données en les ramenant entre 0 et 1

Et on calcule la matrice de corrélation pour choisir les variables les plus corrélées au prix

In [55]:
# On cherche maintenant à trouver les variables les plus corrélées avec le prix
# On commence par concaténer les deux jeux de données
XY = pd.concat([Data_X,Data_Y], axis=1)
# Nous normalisons d'abord les données
for column in XY:
    XY[column]= (XY[column] - XY[column].mean()) / XY[column].std()
corr = XY.corr()
Data_X = XY.iloc[:,:-2]
Data_Y = XY.iloc[:,-2:]
# On affiche la matrice de corrélation
'''print(corr)'''
# On affiche la matrice de corrélation sous forme de heatmap
plt.figure()
# Elargir le plot
plt.figure(figsize=(20,10))
plt.matshow(corr)
plt.title("Matrice de corrélation")
plt.show()

In [56]:
# On cherche maintenant à trouver les variables les plus corrélées avec le prix
# On affiche les variables les plus corrélées avec le prix
print("Variables les plus corrélées avec le prix :")
print(np.absolute(corr["TARGET"]).sort_values(ascending=False)[:10])
# On prend les 6 valeurs les plus corellées pour faire une regression linéaire

In [57]:
# DE_NET_IMPORT semble être la variable la plus corrélée au prix, on tente donc de représenter cette corrélation sur un scatter plot
plt.figure()
# Elargir le plot
plt.figure(figsize=(4,4))
plt.scatter(Data_Y["TARGET"],Data_X["DE_WINDPOW"],)
# Mettre les titres sur les axes
plt.xlabel('DE_WINDPOW', fontweight='bold')
plt.ylabel('TARGET', fontweight='bold')
plt.show()

## Visiblement la corrélation semble limités mais nous ne disposons pas d'autres données pour expliquer le prix
### On commence par une regression linéaire simple avec la force du vent en Allemagne (DE_WINDPOW)

Le code effectue une régression linéaire en utilisant le modèle LinearRegression de scikit-learn.

La première ligne crée une instance du modèle de régression linéaire.

La deuxième ligne utilise la fonction train_test_split de scikit-learn pour diviser les données en un ensemble de formation (X_train et Y_train) et un ensemble de test (X_test et Y_test). La taille de l'ensemble de test est de 20% des données, et le paramètre random_state est défini à 101 pour s'assurer que la division des données est la même à chaque fois que le code est exécuté.

La troisième ligne entraîne le modèle de régression linéaire en utilisant les données d'entraînement. Plus précisément, le modèle est entraîné en utilisant les données de la colonne "DE_NET_EXPORT" de la variable prédictive Data_X et les données de la colonne "TARGET" de la variable cible Data_Y.

Enfin, la dernière ligne affiche le coefficient de la pente de la régression linéaire en utilisant la propriété coef_ de l'objet LinearRegression. Ce coefficient représente la relation linéaire entre la variable prédictive et la variable cible.

In [58]:
LR = LinearRegression() # Création d'une instance du modèle de régression linéaire
X_train, X_test, Y_train, Y_test = train_test_split(Data_X,Data_Y, test_size=0.2,random_state=101)
LR.fit(pd.DataFrame(X_train["DE_NET_EXPORT"]),pd.DataFrame(Y_train["TARGET"]))
print(LR.coef_)

Pour la deuxième partie de la régression linéaire,

La première ligne réalise une prédiction à partir du modèle de régression linéaire entraîné, sur les données de l'ensemble de test. Les prédictions sont effectuées en utilisant les données de la colonne "DE_NET_EXPORT" de l'ensemble de test, qui sont passées à la méthode predict du modèle de régression linéaire. Les prédictions sont stockées dans la variable prediction.

La deuxième ligne crée une nouvelle figure de taille 4x4 pouces en utilisant la méthode figure de la bibliothèque Matplotlib.

La troisième ligne crée un nuage de points en utilisant la méthode scatter de la bibliothèque Matplotlib. Les données de l'ensemble de test sont affichées sur l'axe des abscisses et les prédictions correspondantes sont affichées sur l'axe des ordonnées. La variable cible réelle est représentée par les points bleus.

In [59]:
prediction = LR.predict(pd.DataFrame(X_test['DE_NET_EXPORT']))
plt.figure(figsize=(4,4))
plt.scatter(Y_test["TARGET"],prediction)

# Le résultat de la regression linéaire simple n'est pas très convaincant

## Ensuite on essaye avec les 5 variables ayant le plus fort coéfficient de corrélation le plus fort


In [60]:
LR = LinearRegression() # Creating an Instance of LinearRegression model
X_train, X_test, Y_train, Y_test = train_test_split(Data_X,Data_Y, test_size=0.5,random_state=101)
LR.fit(X_train[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Y_train["TARGET"]))
print(LR.coef_)

In [61]:
prediction = LR.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
plt.figure(figsize=(4,4))
plt.scatter(Y_test["TARGET"],prediction)

On n'aboutit pas à des résultats concluants avec une regressionn simple
## La regression RIDGE donne d'excellents résultats pour des variables fortement corrélées, ici elle ne le sont pas (mais le sujet nous l'impose quand même)

In [62]:
ridge = Ridge(alpha=2.177999999999871)
ridge.fit(X_train[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Y_train["TARGET"]))
print(ridge.coef_)
prediction = ridge.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
plt.figure(figsize=(4,4))
plt.scatter(Y_test["TARGET"],prediction)

La première ligne crée une instance du modèle de régression Ridge en fixant le paramètre de régularisation alpha à 1.0.

La deuxième ligne entraîne le modèle de régression Ridge en utilisant les données d'entraînement en utilisant les colonnes "DE_WINDPOW", "DE_NET_EXPORT", "DE_NET_IMPORT", "DE_RESIDUAL_LOAD", et "FR_WINDPOW" de la variable prédictive X_train et les données de la colonne "TARGET" de la variable cible Y_train.

La troisième ligne affiche les coefficients du modèle de régression Ridge en utilisant la propriété coef_ de l'objet Ridge. Ces coefficients représentent les relations linéaires entre les variables prédictives et la variable cible en tenant compte de la régularisation.

La quatrième ligne utilise le modèle entraîné pour effectuer des prédictions sur les données de test en utilisant les mêmes variables prédictives que celles utilisées pour l'entraînement.

La cinquième ligne crée une figure avec une taille de 4x4 pouces en utilisant la fonction figure(figsize=(4,4)) de matplotlib.pyplot.

La sixième ligne affiche un nuage de points représentant les prédictions par rapport aux valeurs réelles de la variable cible en utilisant la fonction scatter(Y_test["TARGET"],prediction) de matplotlib.pyplot.

#Pour le modèle Lasso:

In [63]:
# Création d'une instance du modèle de régression Lasso
# alpha correspond au coefficient de régularisation
lasso = Lasso(alpha=0.00432,)
# Entraînement du modèle Lasso sur les données d'entraînement
# en utilisant les colonnes sélectionnées pour les variables prédictives et la variable cible
lasso.fit(X_train[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],
           pd.DataFrame(Y_train["TARGET"]))
# Affichage des coefficients de la régression Lasso
print(lasso.coef_)
# Utilisation du modèle Lasso entraîné pour faire des prédictions sur les données de test
prediction = lasso.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
# Création d'une figure pour afficher les prédictions
plt.figure(figsize=(4,4))
# Affichage d'un nuage de points pour comparer les valeurs cibles et les prédictions
plt.scatter(Y_test["TARGET"], prediction)

In [64]:
  # Pour trouver le meilleur coefficient Alpha pour Ridge et Lasso, on utilise l'algorithme suivant :
best_coef = 0
a = 0.0
best_alpha = 0
while a < 3:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Y_train["TARGET"]))
    prediction = ridge.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
    corr_ridge, pval_ridge = spearmanr(Y_test['TARGET'],prediction)
    if corr_ridge > best_coef:
        best_coef = corr_ridge
        best_alpha = a
    a += 0.001

print(best_coef, best_alpha)
# l'interpreteur proposé par google ne permet pas d'obtenir un a convainquant (faute de puissance de calcule)
# Après une exécution locale de cette algorithme nous trouvons
# a = 2.177999999999871
# De même pour le Lasso

Les méthodes récemment utilisées fonctionnent bien avec une bonne corrélation entre les paramètres, or celles des données dont nous disposons sont très faibles

# Pour les KNN:

In [65]:
# Créer une instance du modèle KNN avec 100 voisins
knn_model = KNeighborsRegressor(n_neighbors=100)
# Entraîner le modèle KNN en utilisant les données d'entraînement
knn_model.fit(X_train[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Y_train["TARGET"]))
# Effectuer des prédictions sur les données de test
prediction = knn_model.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
# Créer une figure de taille 4x4 pouces
plt.figure(figsize=(4,4))
# Afficher le nuage de points des prédictions et des valeurs réelles
plt.scatter(Y_test["TARGET"],prediction)

# La dernière méthode que nous explorons est l'arbre de décision pour la regression



In [66]:
reg= DecisionTreeRegressor(random_state=0)
reg.fit(X_train[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Y_train["TARGET"]))
prediction = reg.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
plt.figure(figsize=(4,4))
plt.scatter(Y_test["TARGET"],prediction)

# On cherche maintenant à évaluer les différents modèles testés
- la corrélation de Spearman,
- le coefficient de détermination R2
- l’erreur quadratique moyenne (RMSE)

In [67]:
# Evaluation de la regression linéaire
print("\nPour la regression linéaire :")
LR_predictions = LR.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']])) #Prédiction sur les données utilisées
corr, pval = spearmanr(Y_test['TARGET'], LR_predictions) #Calcul de la corrélation de Spearman et de sa p-value
print("Corrélation de Spearman:", corr)
r2 = r2_score(Y_test['TARGET'], LR_predictions)  #Calcul du coefficient de détermination R2 pour les prédictions
print("Coefficient de détermination R2 :", r2)
rmse = np.sqrt(mean_squared_error(Y_test['TARGET'], LR_predictions)) #Calcul de la racine carrée de l'erreur quadratique moyenne (RMSE) pour les prédictions
print("RMSE:", rmse)
print("P-value:", pval)

# Evaluation de la regression avec Ridge
print("\nPour la regression avec Ridge :")
ridge_predictions = ridge.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']])) #Prédiction sur les données utilisées
corr_ridge, pval_ridge = spearmanr(Y_test['TARGET'], ridge_predictions) #Calcul de la corrélation de Spearman et de sa p-value
print("Corrélation de Spearman:", corr_ridge)
r2_ridge = r2_score(Y_test['TARGET'], ridge_predictions) #Calcul du coefficient de détermination R2
print("Coefficient de détermination R2 :", r2)
rmse_ridge = np.sqrt(mean_squared_error(Y_test['TARGET'], ridge_predictions)) #Calcul de la racine carrée de l'erreur quadratique moyenne (RMSE)
print("RMSE:", rmse_ridge)
print("P-value:", pval_ridge)

# Evaluation de la regression avec Lasso
print("\nPour la regression avec Lasso :")
lasso_predictions = lasso.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']])) #Prédiction sur les données utilisées
corr_lasso, pval_lasso = spearmanr(Y_test['TARGET'], lasso_predictions) #Calcul de la corrélation de Spearman et de sa p-value
print("Corrélation de Spearman:", corr_lasso)
r2_lasso = r2_score(Y_test['TARGET'], lasso_predictions) #Calcul du coefficient de détermination R2
print("Coefficient de détermination R2 :", r2_lasso)
rmse_lasso = np.sqrt(mean_squared_error(Y_test['TARGET'], lasso_predictions)) #Calcul de la racine carrée de l'erreur quadratique moyenne (RMSE)
print("RMSE:", rmse_lasso)
print("P-value:", pval_lasso)

# Evaluation de KNN
print("\nPour KNN :")
knn_predictions = knn_model.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']])) #Prédiction sur les données utilisées
corr_knn, pval_knn = spearmanr(Y_test['TARGET'], knn_predictions) #Calcul de la corrélation de Spearman et de sa p-value
print("Corrélation de Spearman:", corr_knn)
r2_knn = r2_score(Y_test['TARGET'], knn_predictions) #Calcul du coefficient de détermination R2
print("Coefficient de détermination R2 :", r2_knn)
rmse_knn = np.sqrt(mean_squared_error(Y_test['TARGET'], knn_predictions)) #Calcul de la racine carrée de l'erreur quadratique moyenne (RMSE)
print("RMSE:", rmse_knn)
print("P-value:", pval_knn)

# Evaluation de l'arbre de décision pour la regression
print("\nPour l'arbre de décision pour la regression:")
reg_predictions = reg.predict(pd.DataFrame(X_test[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']])) #Prédiction sur les données utilisées
corr_reg, pval_reg = spearmanr(Y_test['TARGET'], reg_predictions) #Calcul de la corrélation de Spearman et de sa p-value
print("Corrélation de Spearman:", corr_reg)
r2_reg = r2_score(Y_test['TARGET'], reg_predictions) #Calcul du coefficient de détermination R2
print("Coefficient de détermination R2 :", r2_reg)
rmse_reg = np.sqrt(mean_squared_error(Y_test['TARGET'], reg_predictions)) #Calcul de la racine carrée de l'erreur quadratique moyenne (RMSE)
print("RMSE:", rmse_reg)
print("P-value:", pval_reg)

# Affichage des résultats
plt.bar(['LR', 'Ridge', 'Lasso', 'KNN', 'Arbre de décision'], [corr, corr_ridge, corr_lasso, corr_knn, corr_reg])
plt.title("Corrélation de Spearman")
plt.show()
plt.bar(['LR', 'Ridge', 'Lasso', 'KNN', 'Arbre de décision'], [r2, r2_ridge, r2_lasso, r2_knn, r2_reg])
plt.title("Coefficient de détermination R2")
plt.show()
plt.bar(['LR', 'Ridge', 'Lasso', 'KNN', 'Arbre de décision'], [rmse, rmse_ridge, rmse_lasso, rmse_knn, rmse_reg])
plt.title("RMSE")
plt.show()
plt.bar(['LR', 'Ridge', 'Lasso', 'KNN', 'Arbre de décision'], [pval, pval_ridge, pval_lasso, pval_knn, pval_reg])
plt.title("P-value")
plt.show()

On peut observer que suivant les indicateurs P-value et RMSE, le modèle d'arbre de décision donne les meilleurs résultats par rapport à notre jeu de données.
# On réalise donc maintenant la prédiction finale avec les DataNew_X en utilisant l'arbre de décision

In [68]:
# Créer une instance du modèle KNN avec 100 voisins
knn_model = KNeighborsRegressor(n_neighbors=10)
# Entraîner le modèle KNN en utilisant les données d'entraînement
knn_model.fit(Data_X[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Data_Y["TARGET"]))
# Effectuer des prédictions sur les données de test
prediction = knn_model.predict(pd.DataFrame(DataNew_X[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']]))
print(prediction)

# Créer une figure de taille 4x4 pouces
plt.figure(figsize=(4,4))
# Afficher le nuage de points des prédictions et des valeurs réelles
# Générer des données aléatoires pour l'exemple
# Créer l'histogramme
plt.hist(prediction, bins=100)

# Ajouter des labels et un titre
plt.xlabel('Valeurs')
plt.ylabel('Fréquence')
plt.title('Distribution des valeurs')

# Afficher le graphique
plt.show()

# Afficher le score total de notre représentation
score = round(knn_model.score(Data_X[['DE_WINDPOW','DE_NET_EXPORT','DE_NET_IMPORT','DE_RESIDUAL_LOAD','FR_WINDPOW']],pd.DataFrame(Data_Y["TARGET"])) * 100, 1)
print("Score total:",score)

# Conclusion:

Tout au long de cette étude, on se rend compte que puisque nos valeurs sont très peu corrélées dès le départ, le résultat obtenu selon les méthodes utilisées varie radicalement. On a pu voir cela avec la régression linéaire ou la régression de Ridge qui donnait de résultat tout sauf concluant, tandis que le KNN avec l'arbre de décision donne des résultats bien plus concluant.

Ce projet nous a donc permis de nous rendre compte que malgré les similitudes entre toutes les méthodes que nous avons étudiées en cours, certaine sont utiles dans certains cas bien précis. Nous avons ainsi tout au long de ce travail découvert dans quels cas laquelle est plus utile et permet d'avoir une meilleure représentation.

Malheureusement, suite à un manque de communication de la part d'un de membre, le projet n'a pas pu être fini dans le temps imparti. Aussi, le manque de précision dans le sujet a été un problème majeur, nous bloquant pendant une longue partie du projet puisque nous ne savions pas réellement par où commencer.